# Advent of Code 2020 - Puzzle 7

## Import packages

In [1]:
import pandas as pd
from collections import defaultdict

## Functions

In [2]:
def load_data(filepath):
    """load data from filepath"""
    
    # open datafile and read string
    with open(filepath, 'r') as file:
        lines = file.read().splitlines()

    # make storage object
    data = defaultdict(lambda: {})

    # iterate over all ines
    for line in lines:
        
        # some ugly corrections
        line = line.replace('no other bags', '0 other bags')
        line = line.replace(' bags', '')
        line = line.replace(' bag', '')

        # unpack bag into bags
        bigbag, smallbags = line.split(' contain ')

        smallbags = smallbags.strip('.')
        smallbags = smallbags.split(',')

        for smallbag in smallbags:
            smallbag = smallbag.lstrip()
            nr, bag = smallbag.split(' ', 1)

            data[bigbag][bag] = int(nr)

    data = dict(data)

    return pd.DataFrame.from_dict(data, orient='index')

def bag_unwrapper(name, df):
    """iterate column over index"""
        
    # storage object
    bags = pd.Series(dtype='int64')
    
    # ignore bags without entry
    if name in df.columns:

        # subset and dropna
        bags = df[name].dropna()

        # iterate over index
        for bagname in bags.index:
            
            # recursion and append series
            bgss = bag_unwrapper(bagname, df)
            bags = bags.append(bgss)

    # reset dtype to int64
    bags = bags.astype('int64')
            
    return bags

def bag_wrapper(name, number, df):
    """iterate index over column"""
    
    # storage object
    bags = pd.Series(dtype='int64')
    
    # ignore bags within entry
    if name in df.index:
    
        # subset and dropna
        bags = df.loc[name].dropna()

        # iterate over index and value
        for bagname, nr in bags.iteritems():
            
            # recursion and append series
            bgss = bag_wrapper(bagname, nr, df)
            bags = bags.append(bgss)

        # scale result
        bags *= number
        
    # reset dtype to int64
    bags = bags.astype('int64')
    
    return bags

## Main

In [3]:
# load data as dataframe
df = load_data('data/input_puzzle_7.txt')

# unwrap shiny bags and get answer
bags = bag_unwrapper('shiny gold', df).groupby(level=0).sum()
answer = bags.index.size

# print result
print(f'Bag options for shiny gold: {answer}')

Bag options for shiny gold: 233


In [4]:
# load data as dataframe
df = load_data('data/input_puzzle_7.txt')

# wrap shiny bags and get answer
bags = bag_wrapper('shiny gold', 1, df).groupby(level=0).sum()
answer = bags.sum()

# print result
print(f'Nr. of bags to fill a shiny gold: {answer}')

Nr. of bags to fill a shiny gold: 421550
